In [1]:
from pprint import pprint
import pandas as pd
import json

In [2]:
def json_to_dfs(dict_dados):
    dict_linhas = {}
    for tipo_colheita in dict_dados:
        colheita_categorias = dict_dados[tipo_colheita]
        dict_linhas[tipo_colheita] = []
        for categoria in colheita_categorias:
            categoria_itens = colheita_categorias[categoria]
            for nome_item in categoria_itens:
                item = categoria_itens[nome_item]
                item['tipo_colheita'] = tipo_colheita
                item['categoria'] = categoria
                
                dict_linhas[tipo_colheita].append(item)
    
    dict_dfs = {}
    for tipo_colheita in dict_linhas:
        dados_atual = dict_linhas[tipo_colheita]
        df_atual = pd.DataFrame(dados_atual)
        dict_dfs[tipo_colheita] = df_atual
    
    return dict_dfs

In [3]:
with open('dados_completos.json', 'r') as json_dados:
    dict_dados = json.loads(json_dados.read())

dict_dfs = json_to_dfs(dict_dados)
dict_dfs['Crops']

,url_img,custo,moeda_custo,estacoes,xp,ganho_base,moeda_ganho,tempo,tipo_recurso,nome,link,tipo_colheita,categoria
0,https://vignette.wikia.nocookie.net/farmtogeth...,35.0,dinheiro,"[fall, spring, winter]",2.0,60.0,dinheiro,20.0,Vegetables,Carrot,https://farmtogether.fandom.com/wiki/Carrot,Crops,Vegetables
1,https://vignette.wikia.nocookie.net/farmtogeth...,15.0,dinheiro,"[fall, spring, winter, summer]",1.0,30.0,dinheiro,10.0,Vegetables,Lettuce,https://farmtogether.fandom.com/wiki/Lettuce,Crops,Vegetables
2,https://vignette.wikia.nocookie.net/farmtogeth...,100.0,dinheiro,"[spring, summer]",5.0,140.0,dinheiro,45.0,Vegetables,Cabbage,https://farmtogether.fandom.com/wiki/Cabbage,Crops,Vegetables
3,https://vignette.wikia.nocookie.net/farmtogeth...,500.0,dinheiro,"[fall, spring]",65.0,850.0,dinheiro,1080.0,Vegetables,Lombardy Cabbage,https://farmtogether.fandom.com/wiki/Lombardy_...,Crops,Vegetables
4,https://vignette.wikia.nocookie.net/farmtogeth...,4200.0,dinheiro,"[fall, spring, winter]",10.0,4300.0,dinheiro,20.0,Vegetables,Purple Carrot,https://farmtogether.fandom.com/wiki/Purple_Ca...,Crops,Vegetables
...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,https://vignette.wikia.nocookie.net/farmtogeth...,300.0,dinheiro,"[fall, spring, winter, summer]",2.0,1.0,diamante,20.0,No Resource,Toy Pinwheels,https://farmtogether.fandom.com/wiki/Toy_Pinwh...,Crops,Event
93,https://vignette.wikia.nocookie.net/farmtogeth...,800.0,dinheiro,"[fall, spring, winter, summer]",18.0,1.0,diamante,180.0,No Resource,Musical Root,https://farmtogether.fandom.com/wiki/Musical_Root,Crops,Event
94,https://vignette.wikia.nocookie.net/farmtogeth...,500.0,dinheiro,"[fall, spring, winter, summer]",11.0,1.0,diamante,90.0,No Resource,Magic Crystal,https://farmtogether.fandom.com/wiki/Magic_Cry...,Crops,Event
95,https://vignette.wikia.nocookie.net/farmtogeth...,1000.0,dinheiro,"[fall, spring, winter, summer]",22.0,1.0,diamante,240.0,No Resource,Party Candles,https://farmtogether.fandom.com/wiki/Party_Can...,Crops,Event


# Passar por cada df e incluir a coluna de ganho/minuto

## Padronização dos ganho em dinheiros, não em diamantes

### Dict de conversão

In [4]:
DICT_CONVERSAO = {
    'diamante_dinheiro': 1000,
    'fruit_dinheiro': 500,
    'vegetable_dinheiro': 250,
    'grape_dinheiro': 400,
    'flower_dinheiro': 400,
    'cereal_dinheiro': 200,
    'nut_dinheiro': 400,
    'milk_dinheiro': 500,
    'egg_dinheiro': 1000,
    'fish_dinheiro': 400,
    'honey_dinheiro': 5000,
    'fabric_dinheiro': 400,
    'cheese_dinheiro': 10000,
    'jam_dinheiro': 10000,
    'mushroom_dinheiro': 400,
    'spice_dinheiro': float(10000/15)
}

### Tipos de ganhos que podem ser gerados

 - Dinheiro -> Dinheiro: 'dinheiro_dinheiro'
 - Dinheiro -> Diamante: 'dinheiro_diamante'
 - Diamante -> Dinheiro: 'diamante_dinheiro'
 - Diamante -> Diamante: 'diamante_diamante'

## Crops

### Gerando realmente uma medida de valor

In [5]:
df_crops = dict_dfs['Crops'].copy()
CUSTO_BASE_CROPS = 10 #moeda = dinheiros

# Classificar cada item com seu tipo de ganho
df_crops.loc[(df_crops['moeda_custo'] == df_crops['moeda_ganho']) & (df_crops['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_dinheiro'
df_crops.loc[(df_crops['moeda_custo'] != df_crops['moeda_ganho']) & (df_crops['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_diamante'
df_crops.loc[(df_crops['moeda_custo'] != df_crops['moeda_ganho']) & (df_crops['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_dinheiro'
df_crops.loc[(df_crops['moeda_custo'] == df_crops['moeda_ganho']) & (df_crops['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_diamante'

df_crops['ganho_por_min'] = 0
df_crops.loc[((df_crops['tipo_ganho'] == 'dinheiro_dinheiro') | (df_crops['tipo_ganho'] == 'diamante_diamante')), 'ganho_por_min'] = (df_crops['ganho_base'] - (df_crops['custo'] + CUSTO_BASE_CROPS))/df_crops['tempo']

df_crops['xp_por_min'] = 0
df_crops['xp_por_min'] = df_crops['xp']/df_crops['tempo']
df_crops[['nome', 'custo', 'moeda_custo', 'ganho_base', 'xp', 'moeda_ganho', 'tempo', 'ganho_por_min', 'tipo_ganho', 'xp_por_min']].sort_values('xp_por_min', ascending=False)[:10]

,nome,custo,moeda_custo,ganho_base,xp,moeda_ganho,tempo,ganho_por_min,tipo_ganho,xp_por_min
72,Sugar Cane,1400.0,dinheiro,1480.0,50.0,dinheiro,60.0,1.166667,dinheiro_dinheiro,0.833333
28,Jicama,2150.0,dinheiro,2220.0,6.0,dinheiro,10.0,6.000000,dinheiro_dinheiro,0.600000
42,Melon,1000.0,dinheiro,1060.0,5.0,dinheiro,10.0,5.000000,dinheiro_dinheiro,0.500000
4,Purple Carrot,4200.0,dinheiro,4300.0,10.0,dinheiro,20.0,4.500000,dinheiro_dinheiro,0.500000
18,Yellow Zucchini,850.0,dinheiro,920.0,6.0,dinheiro,15.0,4.000000,dinheiro_dinheiro,0.400000
51,Black Goji Berry,2250.0,dinheiro,2400.0,10.0,dinheiro,25.0,5.600000,dinheiro_dinheiro,0.400000
14,Romanesco,6000.0,dinheiro,6500.0,33.0,dinheiro,90.0,5.444444,dinheiro_dinheiro,0.366667
10,Broccoli,5000.0,dinheiro,5250.0,20.0,dinheiro,60.0,4.000000,dinheiro_dinheiro,0.333333
15,Cheddar Cauliflower,2400.0,dinheiro,2600.0,14.0,dinheiro,45.0,4.222222,dinheiro_dinheiro,0.311111
27,Turnip,200.0,dinheiro,225.0,3.0,dinheiro,10.0,1.500000,dinheiro_dinheiro,0.300000


In [6]:
group_recurso = df_crops.groupby('tipo_recurso')
df_fruits = group_recurso.get_group('Fruits')
df_fruits[['nome', 'custo', 'moeda_custo', 'ganho_base', 'xp', 'moeda_ganho', 'tempo', 'ganho_por_min', 'tipo_ganho', 'xp_por_min']].sort_values('ganho_por_min', ascending=False)

,nome,custo,moeda_custo,ganho_base,xp,moeda_ganho,tempo,ganho_por_min,tipo_ganho,xp_por_min
42,Melon,1000.0,dinheiro,1060.0,5.0,dinheiro,10.0,5.000000,dinheiro_dinheiro,0.500000
16,Crookneck Pumpkin,10000.0,dinheiro,12000.0,110.0,dinheiro,600.0,3.316667,dinheiro_dinheiro,0.183333
47,Physalis,8000.0,dinheiro,11000.0,170.0,dinheiro,1440.0,2.076389,dinheiro_dinheiro,0.118056
43,Pineapple,5500.0,dinheiro,7000.0,115.0,dinheiro,960.0,1.552083,dinheiro_dinheiro,0.119792
44,Kiwifruit,1000.0,dinheiro,1600.0,86.0,dinheiro,1440.0,0.409722,dinheiro_dinheiro,0.059722
46,Strawberry,800.0,dinheiro,1500.0,105.0,dinheiro,2160.0,0.319444,dinheiro_dinheiro,0.048611
45,Watermelon,250.0,dinheiro,800.0,95.0,dinheiro,2880.0,0.187500,dinheiro_dinheiro,0.032986


In [7]:
cereal = df_crops['tipo_recurso'] == 'Cereals'

df_crops[['nome', 'ganho_por_min']][cereal].sort_values('ganho_por_min', ascending=False)

,nome,ganho_por_min
62,Red Sorghum,1.733333
61,White Sorghum,1.483333
57,Barley,1.034722
59,Rye,0.861111
60,Corn,0.343750
58,Wheat,0.270833


In [8]:
df_crops['dinheiros_por_diamante'] = 0
df_crops.loc[df_crops['tipo_ganho'] == 'diamante_dinheiro', 'dinheiros_por_diamante'] = df_crops['ganho_base']/df_crops['custo']
df_crops.loc[df_crops['tipo_ganho'] == 'dinheiro_diamante', 'dinheiros_por_diamante'] = (df_crops['custo'] + CUSTO_BASE_CROPS)/df_crops['ganho_base']

diamante_dinheiro = df_crops['tipo_ganho'] == 'diamante_dinheiro'
dinheiro_diamante = df_crops['tipo_ganho'] == 'dinheiro_diamante'
df_crops[diamante_dinheiro | dinheiro_diamante][:5]

,url_img,custo,moeda_custo,estacoes,xp,ganho_base,moeda_ganho,tempo,tipo_recurso,nome,link,tipo_colheita,categoria,tipo_ganho,ganho_por_min,xp_por_min,dinheiros_por_diamante
67,https://vignette.wikia.nocookie.net/farmtogeth...,10000.0,dinheiro,[spring],100.0,15.0,diamante,390.0,Spice,Turmeric,https://farmtogether.fandom.com/wiki/Turmeric,Crops,Spice,dinheiro_diamante,0.0,0.256410,667.333333
68,https://vignette.wikia.nocookie.net/farmtogeth...,500.0,dinheiro,"[fall, summer]",25.0,1.0,diamante,240.0,Spice,Black Peppercorn,https://farmtogether.fandom.com/wiki/Black_Pep...,Crops,Spice,dinheiro_diamante,0.0,0.104167,510.000000
69,https://vignette.wikia.nocookie.net/farmtogeth...,4400.0,dinheiro,"[spring, winter]",65.0,8.0,diamante,360.0,Spice,Pink Peppercorn,https://farmtogether.fandom.com/wiki/Pink_Pepp...,Crops,Spice,dinheiro_diamante,0.0,0.180556,551.250000
70,https://vignette.wikia.nocookie.net/farmtogeth...,2200.0,dinheiro,"[fall, winter]",40.0,4.0,diamante,300.0,Spice,White Peppercorn,https://farmtogether.fandom.com/wiki/White_Pep...,Crops,Spice,dinheiro_diamante,0.0,0.133333,552.500000
75,https://vignette.wikia.nocookie.net/farmtogeth...,1000.0,dinheiro,"[fall, spring, winter, summer]",20.0,2.0,diamante,120.0,Vegetables,Chive,https://farmtogether.fandom.com/wiki/Chive,Crops,DLC,dinheiro_diamante,0.0,0.166667,505.000000


In [9]:
str_json = str(df_crops[['nome',  'custo', 'moeda_custo', 'ganho_base', 'moeda_ganho', 'dinheiros_por_diamante']][diamante_dinheiro | dinheiro_diamante].to_json(orient='records'))
dict_ = {'crop': str_json}
print(dict_)

{'crop': '[{"nome":"Turmeric","custo":10000.0,"moeda_custo":"dinheiro","ganho_base":15.0,"moeda_ganho":"diamante","dinheiros_por_diamante":667.3333333333},{"nome":"Black Peppercorn","custo":500.0,"moeda_custo":"dinheiro","ganho_base":1.0,"moeda_ganho":"diamante","dinheiros_por_diamante":510.0},{"nome":"Pink Peppercorn","custo":4400.0,"moeda_custo":"dinheiro","ganho_base":8.0,"moeda_ganho":"diamante","dinheiros_por_diamante":551.25},{"nome":"White Peppercorn","custo":2200.0,"moeda_custo":"dinheiro","ganho_base":4.0,"moeda_ganho":"diamante","dinheiros_por_diamante":552.5},{"nome":"Chive","custo":1000.0,"moeda_custo":"dinheiro","ganho_base":2.0,"moeda_ganho":"diamante","dinheiros_por_diamante":505.0},{"nome":"Ginger","custo":1600.0,"moeda_custo":"dinheiro","ganho_base":3.0,"moeda_ganho":"diamante","dinheiros_por_diamante":536.6666666667},{"nome":"Easter Egg","custo":500.0,"moeda_custo":"dinheiro","ganho_base":1.0,"moeda_ganho":"diamante","dinheiros_por_diamante":510.0},{"nome":"Cursed Pum

In [10]:
df_trees = dict_dfs['Trees'].copy()

# Classificar cada item com seu tipo de ganho
df_trees.loc[(df_trees['moeda_custo'] == df_trees['moeda_ganho']) & (df_trees['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_dinheiro'
df_trees.loc[(df_trees['moeda_custo'] != df_trees['moeda_ganho']) & (df_trees['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_diamante'
df_trees.loc[(df_trees['moeda_custo'] != df_trees['moeda_ganho']) & (df_trees['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_dinheiro'
df_trees.loc[(df_trees['moeda_custo'] == df_trees['moeda_ganho']) & (df_trees['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_diamante'

df_trees['tempo'] = (17*4) / df_trees['quant_estacoes']

df_trees['ganho_por_min'] = 0
df_trees.loc[((df_trees['tipo_ganho'] == 'dinheiro_dinheiro') | (df_trees['tipo_ganho'] == 'diamante_diamante')), 'ganho_por_min'] = df_trees['ganho_base']/df_trees['tempo']

df_trees['dinheiros_por_diamante'] = 0
df_trees.loc[df_trees['tipo_ganho'] == 'diamante_dinheiro', 'dinheiros_por_diamante'] = df_trees['ganho_base']/df_trees['custo']
df_trees.loc[df_trees['tipo_ganho'] == 'dinheiro_diamante', 'dinheiros_por_diamante'] = df_trees['custo']/df_trees['ganho_base']

df_trees['xp_por_min'] = 0
df_trees['xp_por_min'] = df_trees['xp'] / df_trees['tempo']

df_trees['min_sem_lucro'] = 0
df_trees.loc[(df_trees['ganho_por_min'] > 0) & ((df_trees['tipo_ganho'] == 'diamante_diamante') | (df_trees['tipo_ganho'] == 'dinheiro_dinheiro')), 'min_sem_lucro'] = df_trees['custo'] / df_trees['ganho_por_min']

df_trees[['nome', 'custo', 'moeda_custo', 'estacoes', 'ganho_base', 'xp', 'moeda_ganho', 'tempo', 'ganho_por_min', 'tipo_ganho', 'xp_por_min', 'dinheiros_por_diamante', 'min_sem_lucro']].sort_values('dinheiros_por_diamante', ascending=False)[:5]

,nome,custo,moeda_custo,estacoes,ganho_base,xp,moeda_ganho,tempo,ganho_por_min,tipo_ganho,xp_por_min,dinheiros_por_diamante,min_sem_lucro
58,Pistachio Tree,4000.0,dinheiro,"[spring, summer]",1.0,6.0,diamante,34.0,0.0,dinheiro_diamante,0.176471,4000.0,0.0
59,Chestnut Tree,5000.0,dinheiro,[fall],2.0,9.0,diamante,68.0,0.0,dinheiro_diamante,0.132353,2500.0,0.0
57,Flowering Cherry Tree,10000.0,dinheiro,[fall],4.0,10.0,diamante,68.0,0.0,dinheiro_diamante,0.147059,2500.0,0.0
56,Candy Tree,2500.0,dinheiro,[winter],1.0,7.0,diamante,68.0,0.0,dinheiro_diamante,0.102941,2500.0,0.0
39,Money Tree,5000.0,dinheiro,[winter],2.0,12.0,diamante,68.0,0.0,dinheiro_diamante,0.176471,2500.0,0.0


In [11]:
df_flowers = dict_dfs['Flowers'].copy()

# Classificar cada item com seu tipo de ganho
df_flowers.loc[(df_flowers['moeda_custo'] == df_flowers['moeda_ganho']) & (df_flowers['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_dinheiro'
df_flowers.loc[(df_flowers['moeda_custo'] != df_flowers['moeda_ganho']) & (df_flowers['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_diamante'
df_flowers.loc[(df_flowers['moeda_custo'] != df_flowers['moeda_ganho']) & (df_flowers['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_dinheiro'
df_flowers.loc[(df_flowers['moeda_custo'] == df_flowers['moeda_ganho']) & (df_flowers['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_diamante'

df_flowers['ganho_por_min'] = 0
df_flowers.loc[((df_flowers['tipo_ganho'] == 'dinheiro_dinheiro') | (df_flowers['tipo_ganho'] == 'diamante_diamante')), 'ganho_por_min'] = df_flowers['ganho_base']/df_flowers['tempo']

df_flowers['dinheiros_por_diamante'] = 0
df_flowers.loc[df_flowers['tipo_ganho'] == 'diamante_dinheiro', 'dinheiros_por_diamante'] = df_flowers['ganho_base']/df_flowers['custo']
df_flowers.loc[df_flowers['tipo_ganho'] == 'dinheiro_diamante', 'dinheiros_por_diamante'] = df_flowers['custo']/df_flowers['ganho_base']

df_flowers['xp_por_min'] = 0
df_flowers['xp_por_min'] = df_flowers['xp'] / df_flowers['tempo']

df_flowers['min_sem_lucro'] = 0
df_flowers.loc[(df_flowers['ganho_por_min'] > 0) & ((df_flowers['tipo_ganho'] == 'diamante_diamante') | (df_flowers['tipo_ganho'] == 'dinheiro_dinheiro')), 'min_sem_lucro'] = df_flowers['custo'] / df_flowers['ganho_por_min']

df_flowers[['nome', 'custo', 'moeda_custo', 'ganho_base', 'xp', 'moeda_ganho', 'tempo', 'ganho_por_min', 'tipo_ganho', 'xp_por_min', 'dinheiros_por_diamante', 'min_sem_lucro']].sort_values('ganho_por_min', ascending=False)[:5]

,nome,custo,moeda_custo,ganho_base,xp,moeda_ganho,tempo,ganho_por_min,tipo_ganho,xp_por_min,dinheiros_por_diamante,min_sem_lucro
38,Yellow Wisteria,32000.0,dinheiro,3000.0,170.0,dinheiro,300.0,10.000000,dinheiro_dinheiro,0.566667,0.0,3200.000000
37,Blue Wisteria,16500.0,dinheiro,2500.0,150.0,dinheiro,300.0,8.333333,dinheiro_dinheiro,0.500000,0.0,1980.000000
22,Purple Daffodil,14000.0,dinheiro,500.0,55.0,dinheiro,60.0,8.333333,dinheiro_dinheiro,0.916667,0.0,1680.000000
43,Venus Flytrap,10000.0,dinheiro,1200.0,85.0,dinheiro,150.0,8.000000,dinheiro_dinheiro,0.566667,0.0,1250.000000
35,Purple Bellflower,35000.0,dinheiro,5500.0,290.0,dinheiro,720.0,7.638889,dinheiro_dinheiro,0.402778,0.0,4581.818182


In [12]:
df_ponds = dict_dfs['Ponds'].copy()
df_ponds[:1]

,url_img,custo,moeda_custo,xp,ganho_grande,moeda_ganho,ganho_pequeno,tempo,p_pesca,p_ganho_grande,nome,link,tipo_colheita,categoria
0,https://vignette.wikia.nocookie.net/farmtogeth...,200.0,dinheiro,3.0,30.0,dinheiro,3.0,30,0.5,0.2,Tropical Fish,https://farmtogether.fandom.com/wiki/Tropical_...,Ponds,Pond overview


In [13]:
df_ponds = dict_dfs['Ponds'].copy()

# Classificar cada item com seu tipo de ganho
df_ponds.loc[(df_ponds['moeda_custo'] == df_ponds['moeda_ganho']) & (df_ponds['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_dinheiro'
df_ponds.loc[(df_ponds['moeda_custo'] != df_ponds['moeda_ganho']) & (df_ponds['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_diamante'
df_ponds.loc[(df_ponds['moeda_custo'] != df_ponds['moeda_ganho']) & (df_ponds['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_dinheiro'
df_ponds.loc[(df_ponds['moeda_custo'] == df_ponds['moeda_ganho']) & (df_ponds['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_diamante'

df_ponds['ganho_base'] = df_ponds['p_pesca'] * (df_ponds['ganho_grande'] + df_ponds['ganho_pequeno'])

df_ponds['ganho_por_min'] = 0
df_ponds.loc[((df_ponds['tipo_ganho'] == 'dinheiro_dinheiro') | (df_ponds['tipo_ganho'] == 'diamante_diamante')), 'ganho_por_min'] = df_ponds['ganho_base'] / (df_ponds['tempo'] * 2)

df_ponds['dinheiros_por_diamante'] = 0
df_ponds.loc[df_ponds['tipo_ganho'] == 'diamante_dinheiro', 'dinheiros_por_diamante'] = df_ponds['ganho_base'] / df_ponds['custo']

df_ponds.loc[df_ponds['tipo_ganho'] == 'dinheiro_diamante', 'dinheiros_por_diamante'] = df_ponds['custo'] / df_ponds['ganho_base']

df_ponds['xp_por_min'] = 0
df_ponds['xp_por_min'] = df_ponds['xp'] / df_ponds['tempo']

df_ponds['min_sem_lucro'] = 0
df_ponds.loc[(df_ponds['ganho_por_min'] > 0) & ((df_ponds['tipo_ganho'] == 'diamante_diamante') | (df_ponds['tipo_ganho'] == 'dinheiro_dinheiro')), 'min_sem_lucro'] = df_ponds['custo'] / df_ponds['ganho_base']

df_ponds['tipo_recurso'] = 'Fishs'

df_ponds[['nome', 'custo', 'moeda_custo', 'ganho_base', 'xp', 'moeda_ganho', 'tempo', 'ganho_por_min', 'tipo_ganho', 'xp_por_min', 'dinheiros_por_diamante', 'min_sem_lucro']].sort_values('ganho_por_min', ascending=False)[:5]

,nome,custo,moeda_custo,ganho_base,xp,moeda_ganho,tempo,ganho_por_min,tipo_ganho,xp_por_min,dinheiros_por_diamante,min_sem_lucro
24,Perch,30000.0,dinheiro,990.0,33.0,dinheiro,120,4.125000,dinheiro_dinheiro,0.275000,0.0,30.303030
26,Red Sea Bream,50000.0,dinheiro,1375.0,49.0,dinheiro,180,3.819444,dinheiro_dinheiro,0.272222,0.0,36.363636
23,Red Snapper,25000.0,dinheiro,330.0,18.0,dinheiro,45,3.666667,dinheiro_dinheiro,0.400000,0.0,75.757576
21,Sturgeon,17500.0,dinheiro,330.0,19.0,dinheiro,45,3.666667,dinheiro_dinheiro,0.422222,0.0,53.030303
22,Oyster,20000.0,dinheiro,528.0,24.0,dinheiro,75,3.520000,dinheiro_dinheiro,0.320000,0.0,37.878788


In [14]:
df_animals = dict_dfs['Animals'].copy()
df_animals[:1]

,url_img,custo,moeda_custo,xp,ganho_total,moeda_ganho,tempo,tipo_recurso,comida_custo,comida_tempo,comida_vezes,comida_total,nome,link,tipo_colheita,categoria
0,https://vignette.wikia.nocookie.net/farmtogeth...,250.0,dinheiro,855.0,855.0,dinheiro,30.0,Eggs,20,10,3.0,60.0,Leghorn Chicken,https://farmtogether.fandom.com/wiki/Leghorn_C...,Animals,Chicken


In [20]:
df_animals = dict_dfs['Animals'].copy()

# Classificar cada item com seu tipo de ganho
df_animals.loc[(df_animals['moeda_custo'] == df_animals['moeda_ganho']) & (df_animals['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_dinheiro'
df_animals.loc[(df_animals['moeda_custo'] != df_animals['moeda_ganho']) & (df_animals['moeda_custo'] == 'dinheiro'), 'tipo_ganho'] = 'dinheiro_diamante'
df_animals.loc[(df_animals['moeda_custo'] != df_animals['moeda_ganho']) & (df_animals['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_dinheiro'
df_animals.loc[(df_animals['moeda_custo'] == df_animals['moeda_ganho']) & (df_animals['moeda_custo'] == 'diamante'), 'tipo_ganho'] = 'diamante_diamante'

df_animals['ganho_base'] = df_animals['ganho_total'] - df_animals['comida_total']

df_animals['ganho_por_min'] = 0
df_animals.loc[((df_animals['tipo_ganho'] == 'dinheiro_dinheiro') | (df_animals['tipo_ganho'] == 'diamante_diamante')), 'ganho_por_min'] = df_animals['ganho_base']/df_animals['tempo']

df_animals['dinheiros_por_diamante'] = 0
df_animals.loc[df_animals['tipo_ganho'] == 'diamante_dinheiro', 'dinheiros_por_diamante'] = df_animals['ganho_base']/df_animals['custo']
df_animals.loc[df_animals['tipo_ganho'] == 'dinheiro_diamante', 'dinheiros_por_diamante'] = df_animals['custo']/df_animals['ganho_base']

df_animals['xp_por_min'] = 0
df_animals['xp_por_min'] = df_animals['xp'] / df_animals['tempo']

df_animals['min_sem_lucro'] = 0
df_animals.loc[(df_animals['ganho_por_min'] > 0) & ((df_animals['tipo_ganho'] == 'diamante_diamante') | (df_animals['tipo_ganho'] == 'dinheiro_dinheiro')), 'min_sem_lucro'] = df_animals['custo'] / df_animals['ganho_base']

#df_animals[['nome', 'custo', 'moeda_custo', 'ganho_base', 'xp', 'moeda_ganho', 'tempo', 'ganho_por_min', 'tipo_ganho', 'xp_por_min', 'dinheiros_por_diamante', 'min_sem_lucro']].sort_values('ganho_por_min', ascending=False)[:5]
df_animals[['custo', 'moeda_custo', 'xp', 'ganho_total', 'moeda_ganho', 'tempo', 'tipo_recurso', 'comida_custo', 'comida_tempo', 'comida_vezes', 'comida_total', 'nome', 'tipo_ganho', 'ganho_base', 'ganho_por_min', 'dinheiros_por_diamante', 'xp_por_min', 'min_sem_lucro']][:5].sort_values('xp', ascending=False)

,custo,moeda_custo,xp,ganho_total,moeda_ganho,tempo,tipo_recurso,comida_custo,comida_tempo,comida_vezes,comida_total,nome,tipo_ganho,ganho_base,ganho_por_min,dinheiros_por_diamante,xp_por_min,min_sem_lucro
1,1300.0,dinheiro,60014.0,60014.0,dinheiro,60.0,Eggs,130,15,4.0,520.0,New Hampshire Chicken,dinheiro_dinheiro,59494.0,991.566667,0.0,1000.233333,0.021851
0,250.0,dinheiro,855.0,855.0,dinheiro,30.0,Eggs,20,10,3.0,60.0,Leghorn Chicken,dinheiro_dinheiro,795.0,26.500000,0.0,28.500000,0.314465
4,15500.0,dinheiro,29.0,12802.9,dinheiro,60.0,Eggs,3000,15,4.0,12000.0,Barnevelder Chicken,dinheiro_dinheiro,802.9,13.381667,0.0,0.483333,19.305019
3,7000.0,dinheiro,24.0,4352.4,dinheiro,60.0,Eggs,1000,15,4.0,4000.0,Jersey Giant Chicken,dinheiro_dinheiro,352.4,5.873333,0.0,0.400000,19.863791
2,3200.0,dinheiro,19.0,2281.9,dinheiro,60.0,Eggs,500,15,4.0,2000.0,Plymouth Rock Chicken,dinheiro_dinheiro,281.9,4.698333,0.0,0.316667,11.351543
